In [ ]:
from google.colab import auth
auth.authenticate_user()

# Import BigQuery client
from google.cloud import bigquery
client = bigquery.Client(project='mimic-ventilation-project')

In [ ]:
query = """
SELECT itemid, label, category
FROM `physionet-data.mimiciv_icu.d_items`
WHERE LOWER(label) LIKE '%airway type%'
   OR LOWER(label) LIKE '%fio2%'
   OR LOWER(label) LIKE '%ventilator mode%'
   OR LOWER(label) LIKE '%flow rate%'
   OR LOWER(label) LIKE '%peep%'
   OR LOWER(label) LIKE '%tidal volume%'
   OR LOWER(label) LIKE '%peak insp. pressure%'
   OR LOWER(label) LIKE '%plateau pressure%'
   OR LOWER(label) LIKE '%mean airway pressure%'
   OR LOWER(label) LIKE '%minute volume%'
   OR LOWER(label) LIKE '%driving pressure%'
   OR LOWER(label) LIKE '%respiratory rate%'
   OR LOWER(label) LIKE '%psv%'
   OR LOWER(label) LIKE '%pcv%'
   OR LOWER(label) LIKE '%aprv%'
   OR LOWER(label) LIKE '%recruitment%'
   OR LOWER(label) LIKE '%sbt%'
   OR LOWER(label) LIKE '%inspiratory time%'
   OR LOWER(label) LIKE '%expiratory time%'
   OR LOWER(label) LIKE '%sputum%'
   OR LOWER(label) LIKE '%PF%'
   OR LOWER(label) LIKE '%ETCO2%'
   OR LOWER(label) LIKE '%invasive ventilation%'
   OR LOWER(label) LIKE '%intubation%'
   OR LOWER(label) LIKE '%extubation%'
"""

d_items_df = client.query(query).to_dataframe()
print(d_items_df)

     itemid                                         label  \
0    228866                       Plateau Pressure (IABP)   
1    224385                                    Intubation   
2    225451                                Sputum Culture   
3    225468      Unplanned Extubation (patient-initiated)   
4    225477  Unplanned Extubation (non-patient initiated)   
..      ...                                           ...   
107  229238                                FiO2 Challenge   
108  229239                         FiO2 Challenge Result   
109  226754                             FiO2ApacheIIValue   
110  227009                             FiO2_ApacheIV_old   
111  227010                                 FiO2_ApacheIV   

                    category  
0                       IABP  
1    1-Intubation/Extubation  
2                 6-Cultures  
3    1-Intubation/Extubation  
4    1-Intubation/Extubation  
..                       ...  
107             Hemodynamics  
108             Hem

In [ ]:
ventilation_query = """
SELECT subject_id, hadm_id, stay_id, starttime, endtime, value, itemid, patientweight
FROM `physionet-data.mimiciv_icu.procedureevents`
WHERE ordercategoryname = 'Ventilation'
"""

# Run query and convert to DataFrame
ventilation_df = client.query(ventilation_query).to_dataframe()
print(f"Number of ventilation rows: {len(ventilation_df)}")
ventilation_df.head()

Number of ventilation rows: 38717


,subject_id,hadm_id,stay_id,starttime,endtime,value,itemid,patientweight
0,10001884,26184834,37510196,2131-01-11 04:40:00,2131-01-12 17:40:00,2220.0,225792,65.0
1,10001884,26184834,37510196,2131-01-13 04:00:00,2131-01-19 17:45:00,9465.0,225792,65.0
2,10001884,26184834,37510196,2131-01-15 04:07:00,2131-01-19 17:43:00,6576.0,225792,65.0
3,10002013,23581541,39060235,2160-05-18 14:19:00,2160-05-18 18:01:00,222.0,225792,96.0
4,10002114,27793700,34672098,2162-02-17 23:01:00,2162-02-19 18:30:00,2609.0,225792,64.1


In [ ]:
invasiveventilation_query = """
SELECT subject_id, hadm_id, stay_id, starttime, endtime, value, itemid, patientweight
FROM `physionet-data.mimiciv_icu.procedureevents`
WHERE itemid = 225792
"""

# Run query and convert to DataFrame
invasiveventilation_df = client.query(invasiveventilation_query).to_dataframe()
invasiveventilation_df = invasiveventilation_df.drop_duplicates(subset=['starttime', 'endtime'])
invasiveventilation_df = invasiveventilation_df.reset_index(drop=True)
print(f"Number of invasive ventilation rows: {len(invasiveventilation_df)}")
invasiveventilation_df.head()

Number of invasive ventilation rows: 35464


,subject_id,hadm_id,stay_id,starttime,endtime,value,itemid,patientweight
0,10002013,23581541,39060235,2160-05-18 14:19:00,2160-05-18 18:01:00,222.0,225792,96.0
1,10002114,27793700,34672098,2162-02-17 23:01:00,2162-02-19 18:30:00,2609.0,225792,64.1
2,10002428,23473524,35479615,2156-05-11 16:05:00,2156-05-20 10:45:00,12640.0,225792,48.4
3,10002428,28662225,38875437,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,225792,43.0
4,10002667,23197839,31573075,2187-02-24 12:55:00,2187-02-24 16:16:00,201.0,225792,87.7


In [ ]:
unique_invasiveventilation = invasiveventilation_df.shape[0]

print(f"There are {unique_invasiveventilation} invasive ventilation events")

There are 35464 invasive ventilation events


In [ ]:
intubation_query = """
SELECT subject_id, hadm_id, stay_id, starttime, endtime, value, itemid, patientweight
FROM `physionet-data.mimiciv_icu.procedureevents`
WHERE itemid = 224385
"""

# Run query and convert to DataFrame
intubation_df = client.query(intubation_query).to_dataframe()
intubation_df = intubation_df.drop_duplicates(subset=['starttime', 'endtime'])
intubation_df = intubation_df.reset_index(drop=True)
print(f"Number of intubation rows: {len(intubation_df)}")
intubation_df.head()

Number of intubation rows: 9721


,subject_id,hadm_id,stay_id,starttime,endtime,value,itemid,patientweight
0,10002114,27793700,34672098,2162-02-17 23:01:00,2162-02-17 23:02:00,1.0,224385,64.1
1,10003400,23559586,38383343,2137-08-17 21:21:00,2137-08-17 21:22:00,1.0,224385,90.5
2,10003637,28317408,32824762,2150-05-20 07:45:00,2150-05-20 07:46:00,1.0,224385,88.0
3,10004401,27939719,31202136,2144-04-11 05:03:00,2144-04-11 05:04:00,1.0,224385,120.0
4,10004401,29988601,32773003,2144-01-27 19:00:00,2144-01-27 19:01:00,1.0,224385,76.0


In [ ]:
unique_intubation = intubation_df.shape[0]

print(f"There are {unique_intubation} intubation events")

There are 9721 intubation events


In [ ]:
extubation_query = """
SELECT subject_id, hadm_id, stay_id, starttime, endtime, value, itemid, patientweight
FROM `physionet-data.mimiciv_icu.procedureevents`
WHERE itemid IN (227194)
"""

# Run query and convert to DataFrame
extubation_df = client.query(extubation_query).to_dataframe()
extubation_df = extubation_df.drop_duplicates(subset=['starttime', 'endtime'])
extubation_df = extubation_df.reset_index(drop=True)
print(f"Number of extubation rows: {len(extubation_df)}")
extubation_df.head()

Number of extubation rows: 25373


,subject_id,hadm_id,stay_id,starttime,endtime,value,itemid,patientweight
0,10002013,23581541,39060235,2160-05-18 18:00:00,2160-05-18 18:01:00,1.0,227194,96.0
1,10002428,23473524,35479615,2156-05-20 10:45:00,2156-05-20 10:46:00,1.0,227194,48.4
2,10002428,28662225,38875437,2156-04-22 17:10:00,2156-04-22 17:11:00,1.0,227194,43.0
3,10002667,23197839,31573075,2187-02-24 16:16:00,2187-02-24 16:17:00,1.0,227194,87.7
4,10003400,23559586,38383343,2137-08-21 15:40:00,2137-08-21 15:41:00,1.0,227194,99.6


In [ ]:
unique_extubation = extubation_df.shape[0]

print(f"There are {unique_extubation} extubation events")

There are 25373 extubation events


In [ ]:
import pandas as pd

merged_1 = pd.merge(
    intubation_df,
    invasiveventilation_df,
    on=['subject_id', 'stay_id'],
    how = 'outer',
    suffixes=('_int', '_vent')
)

extubation_df_renamed = extubation_df.rename(columns={
    'starttime': 'starttime_ext',
    'endtime': 'endtime_ext',
    'value': 'value_ext',
    'itemid': 'itemid_ext',
    'patientweight': 'patientweight_ext'
})

# merge the result with extubation
ventilation = pd.merge(
    merged_1,
    extubation_df_renamed,
    on=['subject_id', 'stay_id'],
    how = 'outer',
)

In [ ]:
ventilation.head(50)

,subject_id,hadm_id_int,stay_id,starttime_int,endtime_int,value_int,itemid_int,patientweight_int,hadm_id_vent,starttime_vent,endtime_vent,value_vent,itemid_vent,patientweight_vent,hadm_id,starttime_ext,endtime_ext,value_ext,itemid_ext,patientweight_ext
0,10001884,26184834,37510196,2131-01-11 04:30:00,2131-01-11 04:31:00,1.0,224385,65.0,26184834,2131-01-11 04:40:00,2131-01-12 17:40:00,2220.0,225792,65.0,26184834,2131-01-12 17:40:00,2131-01-12 17:41:00,1.0,227194,65.0
1,10001884,26184834,37510196,2131-01-11 04:30:00,2131-01-11 04:31:00,1.0,224385,65.0,26184834,2131-01-13 04:00:00,2131-01-19 17:45:00,9465.0,225792,65.0,26184834,2131-01-12 17:40:00,2131-01-12 17:41:00,1.0,227194,65.0
2,10001884,26184834,37510196,2131-01-11 04:30:00,2131-01-11 04:31:00,1.0,224385,65.0,26184834,2131-01-15 04:07:00,2131-01-19 17:43:00,6576.0,225792,65.0,26184834,2131-01-12 17:40:00,2131-01-12 17:41:00,1.0,227194,65.0
3,10002013,<NA>,39060235,NaT,NaT,NaN,<NA>,NaN,23581541,2160-05-18 14:19:00,2160-05-18 18:01:00,222.0,225792,96.0,23581541,2160-05-18 18:00:00,2160-05-18 18:01:00,1.0,227194,96.0
4,10002114,27793700,34672098,2162-02-17 23:01:00,2162-02-17 23:02:00,1.0,224385,64.1,27793700,2162-02-17 23:01:00,2162-02-19 18:30:00,2609.0,225792,64.1,<NA>,NaT,NaT,NaN,<NA>,NaN
5,10002428,<NA>,35479615,NaT,NaT,NaN,<NA>,NaN,23473524,2156-05-11 16:05:00,2156-05-20 10:45:00,12640.0,225792,48.4,23473524,2156-05-20 10:45:00,2156-05-20 10:46:00,1.0,227194,48.4
6,10002428,<NA>,38875437,NaT,NaT,NaN,<NA>,NaN,28662225,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,225792,43.0,28662225,2156-04-22 17:10:00,2156-04-22 17:11:00,1.0,227194,43.0
7,10002667,<NA>,31573075,NaT,NaT,NaN,<NA>,NaN,23197839,2187-02-24 12:55:00,2187-02-24 16:16:00,201.0,225792,87.7,23197839,2187-02-24 16:16:00,2187-02-24 16:17:00,1.0,227194,87.7
8,10002760,<NA>,31831386,NaT,NaT,NaN,<NA>,NaN,28094813,2141-04-20 14:36:00,2141-04-21 08:00:00,1044.0,225792,91.0,28094813,2141-04-21 08:00:00,2141-04-21 08:01:00,1.0,227194,91.0
9,10003400,<NA>,32128372,NaT,NaT,NaN,<NA>,NaN,20214994,2137-02-25 23:37:00,2137-02-28 14:17:00,3760.0,225792,93.0,<NA>,NaT,NaT,NaN,<NA>,NaN


In [ ]:
ventilation.shape

(44813, 20)

In [ ]:
# Rename columns for final data frame
ventilation = ventilation[['subject_id', 'hadm_id_vent', 'stay_id', 'endtime_int', 'itemid_int', 'starttime_vent', 'endtime_vent', 'value_vent', 'itemid_vent', 'endtime_ext', 'itemid_ext', 'patientweight_ext']]

ventilation = ventilation.rename(columns={
    'hadm_id_vent': 'hadm_id',
    'endtime_int': 'intubation_time',
    'itemid_int': 'intubation_itemid',
    'starttime_vent': 'ventilation_starttime',
    'endtime_vent': 'ventilation_endtime',
    'value_vent': 'ventilation_time',
    'itemid_vent': 'ventilation_itemid',
    'endtime_ext': 'extubation_time',
    'itemid_ext': 'extubation_itemid',
    'patientweight_ext': 'patient_weight'
})

In [ ]:
ventilation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44813 entries, 0 to 44812
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   subject_id             44813 non-null  Int64         
 1   hadm_id                43699 non-null  Int64         
 2   stay_id                44813 non-null  Int64         
 3   intubation_time        18740 non-null  datetime64[us]
 4   intubation_itemid      18740 non-null  Int64         
 5   ventilation_starttime  43699 non-null  datetime64[us]
 6   ventilation_endtime    43699 non-null  datetime64[us]
 7   ventilation_time       43699 non-null  float64       
 8   ventilation_itemid     43699 non-null  Int64         
 9   extubation_time        34440 non-null  datetime64[us]
 10  extubation_itemid      34440 non-null  Int64         
 11  patient_weight         34440 non-null  float64       
dtypes: Int64(6), datetime64[us](4), float64(2)
memory usage: 4.4

In [ ]:
# sort by subject_id, hadm_id, stay_id, and starttime
ventilation = ventilation.sort_values(by=['subject_id', 'hadm_id', 'stay_id', 'intubation_time', 'ventilation_starttime'])

# reset index
ventilation = ventilation.reset_index(drop=True)

# display the first few rows
ventilation.head(50)

,subject_id,hadm_id,stay_id,intubation_time,intubation_itemid,ventilation_starttime,ventilation_endtime,ventilation_time,ventilation_itemid,extubation_time,extubation_itemid,patient_weight
0,10001884,26184834,37510196,2131-01-11 04:31:00,224385,2131-01-11 04:40:00,2131-01-12 17:40:00,2220.0,225792,2131-01-12 17:41:00,227194,65.0
1,10001884,26184834,37510196,2131-01-11 04:31:00,224385,2131-01-13 04:00:00,2131-01-19 17:45:00,9465.0,225792,2131-01-12 17:41:00,227194,65.0
2,10001884,26184834,37510196,2131-01-11 04:31:00,224385,2131-01-15 04:07:00,2131-01-19 17:43:00,6576.0,225792,2131-01-12 17:41:00,227194,65.0
3,10002013,23581541,39060235,NaT,<NA>,2160-05-18 14:19:00,2160-05-18 18:01:00,222.0,225792,2160-05-18 18:01:00,227194,96.0
4,10002114,27793700,34672098,2162-02-17 23:02:00,224385,2162-02-17 23:01:00,2162-02-19 18:30:00,2609.0,225792,NaT,<NA>,NaN
5,10002428,23473524,35479615,NaT,<NA>,2156-05-11 16:05:00,2156-05-20 10:45:00,12640.0,225792,2156-05-20 10:46:00,227194,48.4
6,10002428,28662225,38875437,NaT,<NA>,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,225792,2156-04-22 17:11:00,227194,43.0
7,10002667,23197839,31573075,NaT,<NA>,2187-02-24 12:55:00,2187-02-24 16:16:00,201.0,225792,2187-02-24 16:17:00,227194,87.7
8,10002760,28094813,31831386,NaT,<NA>,2141-04-20 14:36:00,2141-04-21 08:00:00,1044.0,225792,2141-04-21 08:01:00,227194,91.0
9,10003400,20214994,32128372,NaT,<NA>,2137-02-25 23:37:00,2137-02-28 14:17:00,3760.0,225792,NaT,<NA>,NaN


In [ ]:
import numpy as np

# Replace extubation_time with NaN if extubation_time < ventilation_starttime
ventilation.loc[
    ventilation['extubation_time'] < ventilation['ventilation_starttime'],
    'extubation_time'
] = np.nan

In [ ]:
ventilation.head(50)

,subject_id,hadm_id,stay_id,intubation_time,intubation_itemid,ventilation_starttime,ventilation_endtime,ventilation_time,ventilation_itemid,extubation_time,extubation_itemid,patient_weight
0,10001884,26184834,37510196,2131-01-11 04:31:00,224385,2131-01-11 04:40:00,2131-01-12 17:40:00,2220.0,225792,2131-01-12 17:41:00,227194,65.0
1,10001884,26184834,37510196,2131-01-11 04:31:00,224385,2131-01-13 04:00:00,2131-01-19 17:45:00,9465.0,225792,NaT,227194,65.0
2,10001884,26184834,37510196,2131-01-11 04:31:00,224385,2131-01-15 04:07:00,2131-01-19 17:43:00,6576.0,225792,NaT,227194,65.0
3,10002013,23581541,39060235,NaT,<NA>,2160-05-18 14:19:00,2160-05-18 18:01:00,222.0,225792,2160-05-18 18:01:00,227194,96.0
4,10002114,27793700,34672098,2162-02-17 23:02:00,224385,2162-02-17 23:01:00,2162-02-19 18:30:00,2609.0,225792,NaT,<NA>,NaN
5,10002428,23473524,35479615,NaT,<NA>,2156-05-11 16:05:00,2156-05-20 10:45:00,12640.0,225792,2156-05-20 10:46:00,227194,48.4
6,10002428,28662225,38875437,NaT,<NA>,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,225792,2156-04-22 17:11:00,227194,43.0
7,10002667,23197839,31573075,NaT,<NA>,2187-02-24 12:55:00,2187-02-24 16:16:00,201.0,225792,2187-02-24 16:17:00,227194,87.7
8,10002760,28094813,31831386,NaT,<NA>,2141-04-20 14:36:00,2141-04-21 08:00:00,1044.0,225792,2141-04-21 08:01:00,227194,91.0
9,10003400,20214994,32128372,NaT,<NA>,2137-02-25 23:37:00,2137-02-28 14:17:00,3760.0,225792,NaT,<NA>,NaN


In [ ]:
# Identify duplicates of intubation_time within each group, keep first occurrence
duplicates = ventilation.duplicated(subset=['subject_id', 'hadm_id', 'stay_id', 'intubation_time'])

# Replace duplicated intubation_time with NaN
ventilation.loc[duplicates, 'intubation_time'] = np.nan

ventilation.head(50)

,subject_id,hadm_id,stay_id,intubation_time,intubation_itemid,ventilation_starttime,ventilation_endtime,ventilation_time,ventilation_itemid,extubation_time,extubation_itemid,patient_weight
0,10001884,26184834,37510196,2131-01-11 04:31:00,224385,2131-01-11 04:40:00,2131-01-12 17:40:00,2220.0,225792,2131-01-12 17:41:00,227194,65.0
1,10001884,26184834,37510196,NaT,224385,2131-01-13 04:00:00,2131-01-19 17:45:00,9465.0,225792,NaT,227194,65.0
2,10001884,26184834,37510196,NaT,224385,2131-01-15 04:07:00,2131-01-19 17:43:00,6576.0,225792,NaT,227194,65.0
3,10002013,23581541,39060235,NaT,<NA>,2160-05-18 14:19:00,2160-05-18 18:01:00,222.0,225792,2160-05-18 18:01:00,227194,96.0
4,10002114,27793700,34672098,2162-02-17 23:02:00,224385,2162-02-17 23:01:00,2162-02-19 18:30:00,2609.0,225792,NaT,<NA>,NaN
5,10002428,23473524,35479615,NaT,<NA>,2156-05-11 16:05:00,2156-05-20 10:45:00,12640.0,225792,2156-05-20 10:46:00,227194,48.4
6,10002428,28662225,38875437,NaT,<NA>,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,225792,2156-04-22 17:11:00,227194,43.0
7,10002667,23197839,31573075,NaT,<NA>,2187-02-24 12:55:00,2187-02-24 16:16:00,201.0,225792,2187-02-24 16:17:00,227194,87.7
8,10002760,28094813,31831386,NaT,<NA>,2141-04-20 14:36:00,2141-04-21 08:00:00,1044.0,225792,2141-04-21 08:01:00,227194,91.0
9,10003400,20214994,32128372,NaT,<NA>,2137-02-25 23:37:00,2137-02-28 14:17:00,3760.0,225792,NaT,<NA>,NaN


In [ ]:
ventilation.shape[0]

44813

In [ ]:
# load hospital admission outcomes
admissions_query = """
SELECT subject_id, hadm_id, admittime, dischtime, deathtime
FROM `physionet-data.mimiciv_3_1_hosp.admissions`
"""

admissions_discharge = client.query(admissions_query).to_dataframe()

In [ ]:
# merge on subject_id and hadm_id
ventilation_ad = ventilation.merge(
    admissions_discharge,
    on=['subject_id', 'hadm_id'],
    how='left'
)

ventilation_ad.head(50)

,subject_id,hadm_id,stay_id,intubation_time,intubation_itemid,ventilation_starttime,ventilation_endtime,ventilation_time,ventilation_itemid,extubation_time,extubation_itemid,patient_weight,admittime,dischtime,deathtime
0,10001884,26184834,37510196,2131-01-11 04:31:00,224385,2131-01-11 04:40:00,2131-01-12 17:40:00,2220.0,225792,2131-01-12 17:41:00,227194,65.0,2131-01-07 20:39:00,2131-01-20 05:15:00,2131-01-20 05:15:00
1,10001884,26184834,37510196,NaT,224385,2131-01-13 04:00:00,2131-01-19 17:45:00,9465.0,225792,NaT,227194,65.0,2131-01-07 20:39:00,2131-01-20 05:15:00,2131-01-20 05:15:00
2,10001884,26184834,37510196,NaT,224385,2131-01-15 04:07:00,2131-01-19 17:43:00,6576.0,225792,NaT,227194,65.0,2131-01-07 20:39:00,2131-01-20 05:15:00,2131-01-20 05:15:00
3,10002013,23581541,39060235,NaT,<NA>,2160-05-18 14:19:00,2160-05-18 18:01:00,222.0,225792,2160-05-18 18:01:00,227194,96.0,2160-05-18 07:45:00,2160-05-23 13:30:00,NaT
4,10002114,27793700,34672098,2162-02-17 23:02:00,224385,2162-02-17 23:01:00,2162-02-19 18:30:00,2609.0,225792,NaT,<NA>,NaN,2162-02-17 22:32:00,2162-03-04 15:16:00,NaT
5,10002428,23473524,35479615,NaT,<NA>,2156-05-11 16:05:00,2156-05-20 10:45:00,12640.0,225792,2156-05-20 10:46:00,227194,48.4,2156-05-11 14:49:00,2156-05-22 14:16:00,NaT
6,10002428,28662225,38875437,NaT,<NA>,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,225792,2156-04-22 17:11:00,227194,43.0,2156-04-12 14:16:00,2156-04-29 16:26:00,NaT
7,10002667,23197839,31573075,NaT,<NA>,2187-02-24 12:55:00,2187-02-24 16:16:00,201.0,225792,2187-02-24 16:17:00,227194,87.7,2187-02-23 16:01:00,2187-02-28 16:00:00,NaT
8,10002760,28094813,31831386,NaT,<NA>,2141-04-20 14:36:00,2141-04-21 08:00:00,1044.0,225792,2141-04-21 08:01:00,227194,91.0,2141-04-20 07:15:00,2141-04-24 13:31:00,NaT
9,10003400,20214994,32128372,NaT,<NA>,2137-02-25 23:37:00,2137-02-28 14:17:00,3760.0,225792,NaT,<NA>,NaN,2137-02-24 10:00:00,2137-03-19 15:45:00,NaT


In [ ]:
ventilation_ad.shape

(44813, 15)

In [ ]:
# ensure datetime types
ventilation_ad['admittime'] = pd.to_datetime(ventilation_ad['admittime'])
ventilation_ad['dischtime'] = pd.to_datetime(ventilation_ad['dischtime'])
ventilation_ad['deathtime'] = pd.to_datetime(ventilation_ad['deathtime'])

In [ ]:
ventilation_ad['los'] = (ventilation_ad['dischtime'] - ventilation_ad['admittime']).dt.total_seconds() // 60
ventilation_ad['los'] = ventilation_ad['los'].astype('Int64')  # Nullable integer type to preserve NaNs

# Define mortality: 1 if deathtime is present, 0 otherwise
ventilation_ad['mortality'] = ventilation_ad['deathtime'].notna().astype(int)

In [ ]:
ventilation_ad.head(50)

,subject_id,hadm_id,stay_id,intubation_time,intubation_itemid,ventilation_starttime,ventilation_endtime,ventilation_time,ventilation_itemid,extubation_time,extubation_itemid,patient_weight,admittime,dischtime,deathtime,los,mortality
0,10001884,26184834,37510196,2131-01-11 04:31:00,224385,2131-01-11 04:40:00,2131-01-12 17:40:00,2220.0,225792,2131-01-12 17:41:00,227194,65.0,2131-01-07 20:39:00,2131-01-20 05:15:00,2131-01-20 05:15:00,17796,1
1,10001884,26184834,37510196,NaT,224385,2131-01-13 04:00:00,2131-01-19 17:45:00,9465.0,225792,NaT,227194,65.0,2131-01-07 20:39:00,2131-01-20 05:15:00,2131-01-20 05:15:00,17796,1
2,10001884,26184834,37510196,NaT,224385,2131-01-15 04:07:00,2131-01-19 17:43:00,6576.0,225792,NaT,227194,65.0,2131-01-07 20:39:00,2131-01-20 05:15:00,2131-01-20 05:15:00,17796,1
3,10002013,23581541,39060235,NaT,<NA>,2160-05-18 14:19:00,2160-05-18 18:01:00,222.0,225792,2160-05-18 18:01:00,227194,96.0,2160-05-18 07:45:00,2160-05-23 13:30:00,NaT,7545,0
4,10002114,27793700,34672098,2162-02-17 23:02:00,224385,2162-02-17 23:01:00,2162-02-19 18:30:00,2609.0,225792,NaT,<NA>,NaN,2162-02-17 22:32:00,2162-03-04 15:16:00,NaT,21164,0
5,10002428,23473524,35479615,NaT,<NA>,2156-05-11 16:05:00,2156-05-20 10:45:00,12640.0,225792,2156-05-20 10:46:00,227194,48.4,2156-05-11 14:49:00,2156-05-22 14:16:00,NaT,15807,0
6,10002428,28662225,38875437,NaT,<NA>,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,225792,2156-04-22 17:11:00,227194,43.0,2156-04-12 14:16:00,2156-04-29 16:26:00,NaT,24610,0
7,10002667,23197839,31573075,NaT,<NA>,2187-02-24 12:55:00,2187-02-24 16:16:00,201.0,225792,2187-02-24 16:17:00,227194,87.7,2187-02-23 16:01:00,2187-02-28 16:00:00,NaT,7199,0
8,10002760,28094813,31831386,NaT,<NA>,2141-04-20 14:36:00,2141-04-21 08:00:00,1044.0,225792,2141-04-21 08:01:00,227194,91.0,2141-04-20 07:15:00,2141-04-24 13:31:00,NaT,6136,0
9,10003400,20214994,32128372,NaT,<NA>,2137-02-25 23:37:00,2137-02-28 14:17:00,3760.0,225792,NaT,<NA>,NaN,2137-02-24 10:00:00,2137-03-19 15:45:00,NaT,33465,0


In [ ]:
ventilation_ad['died_on_vent'] = ((ventilation_ad['extubation_time'].isna()) &
                                  (ventilation_ad['deathtime'].notna())).astype(int)

In [ ]:
ventilation_ad.head(50)

,subject_id,hadm_id,stay_id,intubation_time,intubation_itemid,ventilation_starttime,ventilation_endtime,ventilation_time,ventilation_itemid,extubation_time,extubation_itemid,patient_weight,admittime,dischtime,deathtime,los,mortality,died_on_vent
0,10001884,26184834,37510196,2131-01-11 04:31:00,224385,2131-01-11 04:40:00,2131-01-12 17:40:00,2220.0,225792,2131-01-12 17:41:00,227194,65.0,2131-01-07 20:39:00,2131-01-20 05:15:00,2131-01-20 05:15:00,17796,1,0
1,10001884,26184834,37510196,NaT,224385,2131-01-13 04:00:00,2131-01-19 17:45:00,9465.0,225792,NaT,227194,65.0,2131-01-07 20:39:00,2131-01-20 05:15:00,2131-01-20 05:15:00,17796,1,1
2,10001884,26184834,37510196,NaT,224385,2131-01-15 04:07:00,2131-01-19 17:43:00,6576.0,225792,NaT,227194,65.0,2131-01-07 20:39:00,2131-01-20 05:15:00,2131-01-20 05:15:00,17796,1,1
3,10002013,23581541,39060235,NaT,<NA>,2160-05-18 14:19:00,2160-05-18 18:01:00,222.0,225792,2160-05-18 18:01:00,227194,96.0,2160-05-18 07:45:00,2160-05-23 13:30:00,NaT,7545,0,0
4,10002114,27793700,34672098,2162-02-17 23:02:00,224385,2162-02-17 23:01:00,2162-02-19 18:30:00,2609.0,225792,NaT,<NA>,NaN,2162-02-17 22:32:00,2162-03-04 15:16:00,NaT,21164,0,0
5,10002428,23473524,35479615,NaT,<NA>,2156-05-11 16:05:00,2156-05-20 10:45:00,12640.0,225792,2156-05-20 10:46:00,227194,48.4,2156-05-11 14:49:00,2156-05-22 14:16:00,NaT,15807,0,0
6,10002428,28662225,38875437,NaT,<NA>,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,225792,2156-04-22 17:11:00,227194,43.0,2156-04-12 14:16:00,2156-04-29 16:26:00,NaT,24610,0,0
7,10002667,23197839,31573075,NaT,<NA>,2187-02-24 12:55:00,2187-02-24 16:16:00,201.0,225792,2187-02-24 16:17:00,227194,87.7,2187-02-23 16:01:00,2187-02-28 16:00:00,NaT,7199,0,0
8,10002760,28094813,31831386,NaT,<NA>,2141-04-20 14:36:00,2141-04-21 08:00:00,1044.0,225792,2141-04-21 08:01:00,227194,91.0,2141-04-20 07:15:00,2141-04-24 13:31:00,NaT,6136,0,0
9,10003400,20214994,32128372,NaT,<NA>,2137-02-25 23:37:00,2137-02-28 14:17:00,3760.0,225792,NaT,<NA>,NaN,2137-02-24 10:00:00,2137-03-19 15:45:00,NaT,33465,0,0


In [ ]:
# time between ventilation_endtime and deathtime where there is no extubation_time

ventilation_ad['ventend_death'] = np.where(
    ventilation_ad['extubation_time'].isna() & ventilation_ad['deathtime'].notna(),
    (ventilation_ad['deathtime'] - ventilation_ad['ventilation_endtime']).dt.total_seconds() / 60,
    np.nan
)

In [ ]:
ventilation_ad.head(50)

,subject_id,hadm_id,stay_id,intubation_time,intubation_itemid,ventilation_starttime,ventilation_endtime,ventilation_time,ventilation_itemid,extubation_time,extubation_itemid,patient_weight,admittime,dischtime,deathtime,los,mortality,died_on_vent,ventend_death
0,10001884,26184834,37510196,2131-01-11 04:31:00,224385,2131-01-11 04:40:00,2131-01-12 17:40:00,2220.0,225792,2131-01-12 17:41:00,227194,65.0,2131-01-07 20:39:00,2131-01-20 05:15:00,2131-01-20 05:15:00,17796,1,0,NaN
1,10001884,26184834,37510196,NaT,224385,2131-01-13 04:00:00,2131-01-19 17:45:00,9465.0,225792,NaT,227194,65.0,2131-01-07 20:39:00,2131-01-20 05:15:00,2131-01-20 05:15:00,17796,1,1,690.0
2,10001884,26184834,37510196,NaT,224385,2131-01-15 04:07:00,2131-01-19 17:43:00,6576.0,225792,NaT,227194,65.0,2131-01-07 20:39:00,2131-01-20 05:15:00,2131-01-20 05:15:00,17796,1,1,692.0
3,10002013,23581541,39060235,NaT,<NA>,2160-05-18 14:19:00,2160-05-18 18:01:00,222.0,225792,2160-05-18 18:01:00,227194,96.0,2160-05-18 07:45:00,2160-05-23 13:30:00,NaT,7545,0,0,NaN
4,10002114,27793700,34672098,2162-02-17 23:02:00,224385,2162-02-17 23:01:00,2162-02-19 18:30:00,2609.0,225792,NaT,<NA>,NaN,2162-02-17 22:32:00,2162-03-04 15:16:00,NaT,21164,0,0,NaN
5,10002428,23473524,35479615,NaT,<NA>,2156-05-11 16:05:00,2156-05-20 10:45:00,12640.0,225792,2156-05-20 10:46:00,227194,48.4,2156-05-11 14:49:00,2156-05-22 14:16:00,NaT,15807,0,0,NaN
6,10002428,28662225,38875437,NaT,<NA>,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,225792,2156-04-22 17:11:00,227194,43.0,2156-04-12 14:16:00,2156-04-29 16:26:00,NaT,24610,0,0,NaN
7,10002667,23197839,31573075,NaT,<NA>,2187-02-24 12:55:00,2187-02-24 16:16:00,201.0,225792,2187-02-24 16:17:00,227194,87.7,2187-02-23 16:01:00,2187-02-28 16:00:00,NaT,7199,0,0,NaN
8,10002760,28094813,31831386,NaT,<NA>,2141-04-20 14:36:00,2141-04-21 08:00:00,1044.0,225792,2141-04-21 08:01:00,227194,91.0,2141-04-20 07:15:00,2141-04-24 13:31:00,NaT,6136,0,0,NaN
9,10003400,20214994,32128372,NaT,<NA>,2137-02-25 23:37:00,2137-02-28 14:17:00,3760.0,225792,NaT,<NA>,NaN,2137-02-24 10:00:00,2137-03-19 15:45:00,NaT,33465,0,0,NaN


In [ ]:
num_died_on_vent = ventilation_ad['died_on_vent'].sum()
print(f"Number of patients who died while receiving invasive mechanical ventilation: {num_died_on_vent}")

Number of patients who died while receiving invasive mechanical ventilation: 4142


In [ ]:
# Function to remove contained intervals
def remove_contained_intervals(df):
    keep = []
    max_end = pd.Timestamp.min

    for idx, row in df.iterrows():
        start, end = row['ventilation_starttime'], row['ventilation_endtime']
        if start >= max_end:
            keep.append(True)
            max_end = max(max_end, end)
        else:
            if end <= max_end:
                keep.append(False)  # Fully contained interval → remove
            else:
                keep.append(True)
                max_end = max(max_end, end)
    return df[keep]

# Apply per patient
ventilation_ad = (
    ventilation_ad
    .groupby(['subject_id', 'stay_id'], group_keys=False)
    .apply(remove_contained_intervals)
    .reset_index(drop=True)
)

<ipython-input-89-455c85cda032>:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(remove_contained_intervals)


In [ ]:
ventilation_ad.shape

(36437, 19)

In [ ]:
# Excluding patients ventilated for < 24h

import numpy as np

# Define a function to calculate ventilation duration in hours
def calculate_vent_duration(row):
    # Case 1: extubation_time and intubation_time present
    if pd.notnull(row['intubation_time']) and pd.notnull(row['extubation_time']):
        return (row['extubation_time'] - row['intubation_time']).total_seconds() / 60
    # Case 2: intubation_time and deathtime (but no extubation)
    elif pd.notnull(row['intubation_time']) and pd.notnull(row['deathtime']):
        return (row['deathtime'] - row['intubation_time']).total_seconds() / 60
    # Case 3: ventilation_starttime and extubation_time (no intubation_time)
    elif pd.notnull(row['ventilation_starttime']) and pd.notnull(row['extubation_time']):
        return (row['extubation_time'] - row['ventilation_starttime']).total_seconds() / 60
    # Case 4: ventilation_starttime and deathtime (no intubation or extubation)
    elif pd.notnull(row['ventilation_starttime']) and pd.notnull(row['deathtime']):
        return (row['deathtime'] - row['ventilation_starttime']).total_seconds() / 60
    else:
        return np.nan  # No valid time pair

# Apply the function to calculate ventilation duration
ventilation_ad['vent_duration_mins'] = ventilation_ad.apply(calculate_vent_duration, axis=1)

# Filter for rows with ventilation duration >= 24 hours (= 24h*60m)
ventilation_ad_filtered = ventilation_ad[ventilation_ad['vent_duration_mins'] >= 1440]

# reset index
ventilation_ad_filtered = ventilation_ad_filtered.reset_index(drop=True)

# Display number of patients retained
print(f"Patients ventilated >= 24h: {len(ventilation_ad_filtered)}")

Patients ventilated >= 24h: 12646


In [ ]:
ventilation_ad_filtered.head(50)

,subject_id,hadm_id,stay_id,intubation_time,intubation_itemid,ventilation_starttime,ventilation_endtime,ventilation_time,ventilation_itemid,extubation_time,extubation_itemid,patient_weight,admittime,dischtime,deathtime,los,mortality,died_on_vent,ventend_death,vent_duration_mins
0,10001884,26184834,37510196,2131-01-11 04:31:00,224385,2131-01-11 04:40:00,2131-01-12 17:40:00,2220.0,225792,2131-01-12 17:41:00,227194,65.0,2131-01-07 20:39:00,2131-01-20 05:15:00,2131-01-20 05:15:00,17796,1,0,NaN,2230.0
1,10001884,26184834,37510196,NaT,224385,2131-01-13 04:00:00,2131-01-19 17:45:00,9465.0,225792,NaT,227194,65.0,2131-01-07 20:39:00,2131-01-20 05:15:00,2131-01-20 05:15:00,17796,1,1,690.0,10155.0
2,10002428,23473524,35479615,NaT,<NA>,2156-05-11 16:05:00,2156-05-20 10:45:00,12640.0,225792,2156-05-20 10:46:00,227194,48.4,2156-05-11 14:49:00,2156-05-22 14:16:00,NaT,15807,0,0,NaN,12641.0
3,10002428,28662225,38875437,NaT,<NA>,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,225792,2156-04-22 17:11:00,227194,43.0,2156-04-12 14:16:00,2156-04-29 16:26:00,NaT,24610,0,0,NaN,4141.0
4,10003400,23559586,34577403,NaT,<NA>,2137-08-10 20:33:00,2137-08-11 13:53:00,1040.0,225792,NaT,<NA>,NaN,2137-08-04 00:07:00,2137-09-02 17:05:00,2137-09-02 17:05:00,42778,1,1,31872.0,32912.0
5,10003400,23559586,38383343,2137-08-17 21:22:00,224385,2137-08-17 21:21:00,2137-08-21 15:00:00,5379.0,225792,2137-08-21 15:41:00,227194,99.6,2137-08-04 00:07:00,2137-09-02 17:05:00,2137-09-02 17:05:00,42778,1,0,NaN,5419.0
6,10003637,28317408,32824762,NaT,224385,2150-05-20 07:45:00,2150-05-22 18:39:00,3534.0,225792,NaT,227194,88.0,2150-05-14 19:51:00,2150-05-22 16:25:00,2150-05-22 16:25:00,11314,1,1,-134.0,3400.0
7,10004235,24181354,34100191,NaT,<NA>,2196-02-24 16:52:00,2196-02-27 16:28:00,4296.0,225792,2196-02-27 16:29:00,227194,127.0,2196-02-24 14:38:00,2196-03-04 14:02:00,NaT,12924,0,0,NaN,4297.0
8,10004401,25777141,39699336,NaT,<NA>,2144-06-05 20:45:00,2144-06-18 16:12:00,18447.0,225792,NaT,<NA>,NaN,2144-06-05 19:45:00,2144-06-18 21:30:00,2144-06-18 21:30:00,18825,1,1,318.0,18765.0
9,10004606,29242151,30213599,NaT,<NA>,2159-02-20 18:04:00,2159-02-22 11:04:00,2460.0,225792,2159-02-22 11:04:00,227194,55.0,2159-02-20 13:43:00,2159-03-06 16:51:00,NaT,20348,0,0,NaN,2460.0


In [ ]:
# Excluding patients <18 years and >89 years old at time of admission (all patients > 89 have anchor age set to 91, regardless of how old they were)

# load hospital patients table
patients_query = """
SELECT subject_id, gender, anchor_age
FROM `physionet-data.mimiciv_3_1_hosp.patients`
"""

patients = client.query(patients_query).to_dataframe()

In [ ]:
ventilation_ad_filtered1 = pd.merge(ventilation_ad_filtered, patients[['subject_id', 'anchor_age']], on='subject_id', how='left')

age_filter = ventilation_ad_filtered1['anchor_age'] < 18
old_age_filter = ventilation_ad_filtered1['anchor_age'] > 91

ventilation_ad_filtered2 = ventilation_ad_filtered1[~(age_filter | old_age_filter)]

In [ ]:
ventilation_ad_filtered2.shape

(12646, 21)

In [ ]:
ventilation_ad_filtered2.head(50)

,subject_id,hadm_id,stay_id,intubation_time,intubation_itemid,ventilation_starttime,ventilation_endtime,ventilation_time,ventilation_itemid,extubation_time,...,patient_weight,admittime,dischtime,deathtime,los,mortality,died_on_vent,ventend_death,vent_duration_mins,anchor_age
0,10001884,26184834,37510196,2131-01-11 04:31:00,224385,2131-01-11 04:40:00,2131-01-12 17:40:00,2220.0,225792,2131-01-12 17:41:00,...,65.0,2131-01-07 20:39:00,2131-01-20 05:15:00,2131-01-20 05:15:00,17796,1,0,NaN,2230.0,68
1,10001884,26184834,37510196,NaT,224385,2131-01-13 04:00:00,2131-01-19 17:45:00,9465.0,225792,NaT,...,65.0,2131-01-07 20:39:00,2131-01-20 05:15:00,2131-01-20 05:15:00,17796,1,1,690.0,10155.0,68
2,10002428,23473524,35479615,NaT,<NA>,2156-05-11 16:05:00,2156-05-20 10:45:00,12640.0,225792,2156-05-20 10:46:00,...,48.4,2156-05-11 14:49:00,2156-05-22 14:16:00,NaT,15807,0,0,NaN,12641.0,80
3,10002428,28662225,38875437,NaT,<NA>,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,225792,2156-04-22 17:11:00,...,43.0,2156-04-12 14:16:00,2156-04-29 16:26:00,NaT,24610,0,0,NaN,4141.0,80
4,10003400,23559586,34577403,NaT,<NA>,2137-08-10 20:33:00,2137-08-11 13:53:00,1040.0,225792,NaT,...,NaN,2137-08-04 00:07:00,2137-09-02 17:05:00,2137-09-02 17:05:00,42778,1,1,31872.0,32912.0,72
5,10003400,23559586,38383343,2137-08-17 21:22:00,224385,2137-08-17 21:21:00,2137-08-21 15:00:00,5379.0,225792,2137-08-21 15:41:00,...,99.6,2137-08-04 00:07:00,2137-09-02 17:05:00,2137-09-02 17:05:00,42778,1,0,NaN,5419.0,72
6,10003637,28317408,32824762,NaT,224385,2150-05-20 07:45:00,2150-05-22 18:39:00,3534.0,225792,NaT,...,88.0,2150-05-14 19:51:00,2150-05-22 16:25:00,2150-05-22 16:25:00,11314,1,1,-134.0,3400.0,57
7,10004235,24181354,34100191,NaT,<NA>,2196-02-24 16:52:00,2196-02-27 16:28:00,4296.0,225792,2196-02-27 16:29:00,...,127.0,2196-02-24 14:38:00,2196-03-04 14:02:00,NaT,12924,0,0,NaN,4297.0,47
8,10004401,25777141,39699336,NaT,<NA>,2144-06-05 20:45:00,2144-06-18 16:12:00,18447.0,225792,NaT,...,NaN,2144-06-05 19:45:00,2144-06-18 21:30:00,2144-06-18 21:30:00,18825,1,1,318.0,18765.0,82
9,10004606,29242151,30213599,NaT,<NA>,2159-02-20 18:04:00,2159-02-22 11:04:00,2460.0,225792,2159-02-22 11:04:00,...,55.0,2159-02-20 13:43:00,2159-03-06 16:51:00,NaT,20348,0,0,NaN,2460.0,64


In [ ]:
ventilation_ad_filtered_3 = ventilation_ad_filtered2.copy()

In [ ]:
# Filter for first admission only, unless readmitted within 48h which counts as the same admission

from datetime import timedelta

# Sort the dataframe by subject_id and ventilation_starttime
ventilation_ad_filtered_3 = ventilation_ad_filtered_3.sort_values(by=['subject_id', 'ventilation_starttime']).reset_index(drop=True)

# List to collect accepted rows
filtered_rows = []

# Group by subject_id
for subject_id, group in ventilation_ad_filtered_3.groupby('subject_id'):
    group = group.reset_index(drop=True)

    # Always keep the first row
    filtered_rows.append(group.iloc[0])

    for i in range(1, len(group)):
        prev_row = group.iloc[i - 1]
        curr_row = group.iloc[i]

        # Use extubation_time if available, else ventilation_endtime
        ref_end_time = prev_row['extubation_time']
        if pd.isna(ref_end_time):
            ref_end_time = prev_row['ventilation_endtime']

        # Consider both intubation_time and ventilation_starttime
        keep = False
        for start_col in ['intubation_time', 'ventilation_starttime']:
            start_time = curr_row[start_col]
            if pd.notna(start_time) and pd.notna(ref_end_time):
                if (start_time - ref_end_time) <= timedelta(hours=48):
                    keep = True
                    break

        if keep:
            filtered_rows.append(curr_row)

# Create a new DataFrame from the filtered rows
ventilation_ad_filtered_3 = pd.DataFrame(filtered_rows).reset_index(drop=True)

In [ ]:
ventilation_ad_filtered_3.head(50)

,subject_id,hadm_id,stay_id,intubation_time,intubation_itemid,ventilation_starttime,ventilation_endtime,ventilation_time,ventilation_itemid,extubation_time,...,patient_weight,admittime,dischtime,deathtime,los,mortality,died_on_vent,ventend_death,vent_duration_mins,anchor_age
0,10001884,26184834,37510196,2131-01-11 04:31:00,224385,2131-01-11 04:40:00,2131-01-12 17:40:00,2220.0,225792,2131-01-12 17:41:00,...,65.0,2131-01-07 20:39:00,2131-01-20 05:15:00,2131-01-20 05:15:00,17796,1,0,NaN,2230.0,68
1,10001884,26184834,37510196,NaT,224385,2131-01-13 04:00:00,2131-01-19 17:45:00,9465.0,225792,NaT,...,65.0,2131-01-07 20:39:00,2131-01-20 05:15:00,2131-01-20 05:15:00,17796,1,1,690.0,10155.0,68
2,10002428,28662225,38875437,NaT,<NA>,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,225792,2156-04-22 17:11:00,...,43.0,2156-04-12 14:16:00,2156-04-29 16:26:00,NaT,24610,0,0,NaN,4141.0,80
3,10003400,23559586,34577403,NaT,<NA>,2137-08-10 20:33:00,2137-08-11 13:53:00,1040.0,225792,NaT,...,NaN,2137-08-04 00:07:00,2137-09-02 17:05:00,2137-09-02 17:05:00,42778,1,1,31872.0,32912.0,72
4,10003637,28317408,32824762,NaT,224385,2150-05-20 07:45:00,2150-05-22 18:39:00,3534.0,225792,NaT,...,88.0,2150-05-14 19:51:00,2150-05-22 16:25:00,2150-05-22 16:25:00,11314,1,1,-134.0,3400.0,57
5,10004235,24181354,34100191,NaT,<NA>,2196-02-24 16:52:00,2196-02-27 16:28:00,4296.0,225792,2196-02-27 16:29:00,...,127.0,2196-02-24 14:38:00,2196-03-04 14:02:00,NaT,12924,0,0,NaN,4297.0,47
6,10004401,25777141,39699336,NaT,<NA>,2144-06-05 20:45:00,2144-06-18 16:12:00,18447.0,225792,NaT,...,NaN,2144-06-05 19:45:00,2144-06-18 21:30:00,2144-06-18 21:30:00,18825,1,1,318.0,18765.0,82
7,10004606,29242151,30213599,NaT,<NA>,2159-02-20 18:04:00,2159-02-22 11:04:00,2460.0,225792,2159-02-22 11:04:00,...,55.0,2159-02-20 13:43:00,2159-03-06 16:51:00,NaT,20348,0,0,NaN,2460.0,64
8,10004720,22081550,35009126,NaT,<NA>,2186-11-12 20:29:00,2186-11-17 14:00:00,6811.0,225792,2186-11-17 14:01:00,...,70.0,2186-11-12 18:01:00,2186-11-17 18:30:00,2186-11-17 18:30:00,7229,1,0,NaN,6812.0,61
9,10004733,27411876,39635619,NaT,<NA>,2174-12-04 12:25:00,2174-12-07 16:20:00,4555.0,225792,2174-12-07 16:21:00,...,112.5,2174-12-04 11:28:00,2174-12-27 14:00:00,NaT,33272,0,0,NaN,4556.0,51


In [ ]:
ventilation_ad_filtered_3.shape

(11564, 21)

In [ ]:
# tidy up dataframe

vent_filtered = ventilation_ad_filtered_3.copy()

# drop columns
columns_to_drop = [
    'intubation_itemid', 'ventilation_itemid',
    'extubation_itemid', 'admittime', 'dischtime'
]
vent_filtered.drop(columns=columns_to_drop, inplace=True)

# rename 'anchor_age' to 'age'
vent_filtered.rename(columns={'anchor_age': 'age'}, inplace=True)

# sort for consistent ordering
vent_filtered.sort_values(by=['subject_id', 'ventilation_starttime'], inplace=True)

# create 'hadm_no' and 'stay_no' (reset per subject)
vent_filtered['hadm_no'] = (
    vent_filtered.groupby('subject_id')['hadm_id']
    .transform(lambda x: pd.factorize(x, sort=True)[0] + 1)
)

vent_filtered['stay_no'] = (
    vent_filtered.groupby(['subject_id', 'hadm_no'])['stay_id']
    .transform(lambda x: pd.factorize(x, sort=True)[0] + 1)
)


# reorder columns — insert hadm_no and stay_no before intubation_time
cols = list(vent_filtered.columns)
# temove 'hadm_no' and 'stay_no' from current position
cols.remove('hadm_no')
cols.remove('stay_no')
# find position of 'intubation_time' and insert before it
intubation_idx = cols.index('intubation_time')
cols = cols[:intubation_idx] + ['hadm_no', 'stay_no'] + cols[intubation_idx:]
vent_filtered = vent_filtered[cols]

# final sort by subject_id, hadm_no, and stay_no
vent_filtered.sort_values(by=['subject_id', 'hadm_no', 'stay_no'], inplace=True)
vent_filtered.reset_index(drop=True, inplace=True)

In [ ]:
vent_filtered.head(50)

,subject_id,hadm_id,stay_id,hadm_no,stay_no,intubation_time,ventilation_starttime,ventilation_endtime,ventilation_time,extubation_time,patient_weight,deathtime,los,mortality,died_on_vent,ventend_death,vent_duration_mins,age
0,10001884,26184834,37510196,1,1,2131-01-11 04:31:00,2131-01-11 04:40:00,2131-01-12 17:40:00,2220.0,2131-01-12 17:41:00,65.0,2131-01-20 05:15:00,17796,1,0,NaN,2230.0,68
1,10001884,26184834,37510196,1,1,NaT,2131-01-13 04:00:00,2131-01-19 17:45:00,9465.0,NaT,65.0,2131-01-20 05:15:00,17796,1,1,690.0,10155.0,68
2,10002428,28662225,38875437,1,1,NaT,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,2156-04-22 17:11:00,43.0,NaT,24610,0,0,NaN,4141.0,80
3,10003400,23559586,34577403,1,1,NaT,2137-08-10 20:33:00,2137-08-11 13:53:00,1040.0,NaT,NaN,2137-09-02 17:05:00,42778,1,1,31872.0,32912.0,72
4,10003637,28317408,32824762,1,1,NaT,2150-05-20 07:45:00,2150-05-22 18:39:00,3534.0,NaT,88.0,2150-05-22 16:25:00,11314,1,1,-134.0,3400.0,57
5,10004235,24181354,34100191,1,1,NaT,2196-02-24 16:52:00,2196-02-27 16:28:00,4296.0,2196-02-27 16:29:00,127.0,NaT,12924,0,0,NaN,4297.0,47
6,10004401,25777141,39699336,1,1,NaT,2144-06-05 20:45:00,2144-06-18 16:12:00,18447.0,NaT,NaN,2144-06-18 21:30:00,18825,1,1,318.0,18765.0,82
7,10004606,29242151,30213599,1,1,NaT,2159-02-20 18:04:00,2159-02-22 11:04:00,2460.0,2159-02-22 11:04:00,55.0,NaT,20348,0,0,NaN,2460.0,64
8,10004720,22081550,35009126,1,1,NaT,2186-11-12 20:29:00,2186-11-17 14:00:00,6811.0,2186-11-17 14:01:00,70.0,2186-11-17 18:30:00,7229,1,0,NaN,6812.0,61
9,10004733,27411876,39635619,1,1,NaT,2174-12-04 12:25:00,2174-12-07 16:20:00,4555.0,2174-12-07 16:21:00,112.5,NaT,33272,0,0,NaN,4556.0,51


In [ ]:
vent_filtered.shape

(11564, 18)

In [ ]:
# Exclude patients on end of life or palliative care

query = """
SELECT itemid, label, category
FROM `physionet-data.mimiciv_icu.d_items`
WHERE LOWER(label) LIKE '%palliative%'
"""

palliative_df = client.query(query).to_dataframe()
print(palliative_df)

   itemid                                    label    category
0  229150  Palliative Care NCP - Expected outcomes  Care Plans
1  229152      Palliative Care NCP - Interventions  Care Plans
2  229154       Palliative Care NCP - Plan revised  Care Plans
3  229155   Palliative Care NCP - Problem resolved  Care Plans


In [ ]:
query = """
SELECT icd_code, icd_version, long_title
FROM `physionet-data.mimiciv_3_1_hosp.d_icd_diagnoses`
WHERE LOWER(long_title) LIKE '%palliative%'
"""

palliative_diag_df = client.query(query).to_dataframe()
print(palliative_diag_df)

  icd_code  icd_version                     long_title
0     V667            9  Encounter for palliative care
1     Z515           10  Encounter for palliative care


In [ ]:
# load hospital diagnoses_icd table
diagnosis_query = """
SELECT subject_id, hadm_id, seq_num, icd_code
FROM `physionet-data.mimiciv_3_1_hosp.diagnoses_icd`
"""

diagnosis = client.query(diagnosis_query).to_dataframe()
diagnosis.head()

,subject_id,hadm_id,seq_num,icd_code
0,10000980,25242409,32,79092
1,10003400,20214994,29,V6441
2,10003400,20214994,32,V8533
3,10004733,27411876,27,E8798
4,10005024,25023471,30,7856


In [ ]:
codes_to_exclude = ['V667', 'Z515']
patients_to_exclude = diagnosis[diagnosis['icd_code'].isin(codes_to_exclude)]

# remove these patients
non_palliative_df = vent_filtered[~vent_filtered['subject_id'].isin(patients_to_exclude)]

non_palliative_df.head()

,subject_id,hadm_id,stay_id,hadm_no,stay_no,intubation_time,ventilation_starttime,ventilation_endtime,ventilation_time,extubation_time,patient_weight,deathtime,los,mortality,died_on_vent,ventend_death,vent_duration_mins,age
0,10001884,26184834,37510196,1,1,2131-01-11 04:31:00,2131-01-11 04:40:00,2131-01-12 17:40:00,2220.0,2131-01-12 17:41:00,65.0,2131-01-20 05:15:00,17796,1,0,NaN,2230.0,68
1,10001884,26184834,37510196,1,1,NaT,2131-01-13 04:00:00,2131-01-19 17:45:00,9465.0,NaT,65.0,2131-01-20 05:15:00,17796,1,1,690.0,10155.0,68
2,10002428,28662225,38875437,1,1,NaT,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,2156-04-22 17:11:00,43.0,NaT,24610,0,0,NaN,4141.0,80
3,10003400,23559586,34577403,1,1,NaT,2137-08-10 20:33:00,2137-08-11 13:53:00,1040.0,NaT,NaN,2137-09-02 17:05:00,42778,1,1,31872.0,32912.0,72
4,10003637,28317408,32824762,1,1,NaT,2150-05-20 07:45:00,2150-05-22 18:39:00,3534.0,NaT,88.0,2150-05-22 16:25:00,11314,1,1,-134.0,3400.0,57


In [ ]:
non_palliative_df.shape

(11564, 18)

In [ ]:
palliative_query = """
SELECT DISTINCT subject_id, stay_id
FROM `physionet-data.mimiciv_icu.chartevents`
WHERE itemid IN (229150, 229152, 229154, 229155)
"""

palliative_patients_df = client.query(palliative_query).to_dataframe()

# Merge to identify patients to exclude
non_palliative_df_filtered = non_palliative_df.merge(
    palliative_patients_df,
    on=['subject_id', 'stay_id'],
    how='left',
    indicator=True
)

# Keep only those not found in the palliative list
non_palliative_df_filtered = non_palliative_df_filtered[non_palliative_df_filtered['_merge'] == 'left_only']

# Drop the merge indicator column
non_palliative_df_filtered = non_palliative_df_filtered.drop(columns=['_merge'])

In [ ]:
non_palliative_df_filtered.shape

(11037, 18)

In [ ]:
non_palliative_df_filtered.head()

,subject_id,hadm_id,stay_id,hadm_no,stay_no,intubation_time,ventilation_starttime,ventilation_endtime,ventilation_time,extubation_time,patient_weight,deathtime,los,mortality,died_on_vent,ventend_death,vent_duration_mins,age
0,10001884,26184834,37510196,1,1,2131-01-11 04:31:00,2131-01-11 04:40:00,2131-01-12 17:40:00,2220.0,2131-01-12 17:41:00,65.0,2131-01-20 05:15:00,17796,1,0,NaN,2230.0,68
1,10001884,26184834,37510196,1,1,NaT,2131-01-13 04:00:00,2131-01-19 17:45:00,9465.0,NaT,65.0,2131-01-20 05:15:00,17796,1,1,690.0,10155.0,68
2,10002428,28662225,38875437,1,1,NaT,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,2156-04-22 17:11:00,43.0,NaT,24610,0,0,NaN,4141.0,80
3,10003400,23559586,34577403,1,1,NaT,2137-08-10 20:33:00,2137-08-11 13:53:00,1040.0,NaT,NaN,2137-09-02 17:05:00,42778,1,1,31872.0,32912.0,72
4,10003637,28317408,32824762,1,1,NaT,2150-05-20 07:45:00,2150-05-22 18:39:00,3534.0,NaT,88.0,2150-05-22 16:25:00,11314,1,1,-134.0,3400.0,57


In [ ]:
vent_outcomes = non_palliative_df_filtered.copy()

In [ ]:
# Ventilation outcomes

# Successful extubation = free from ETT and tracheostomy >= 48h after extubation (Blackwoood et al, doi: 10.1097/CCM.0000000000003904) AND spont ventilation without PS >=48h after extubation AND alive >= 48h after extubation
# Extubation failure = died on vent OR reintubated within 48h OR died within 48h OR required NIV > 48h post-extubation

# First, NIV

query = """
SELECT itemid, label, category
FROM `physionet-data.mimiciv_icu.d_items`
WHERE LOWER(label) LIKE '%non-invasive ventilation%'
OR LOWER(label) LIKE '%cpap%'
OR LOWER(label) LIKE '%bipap%'
"""

niv = client.query(query).to_dataframe()
print(niv)

   itemid                     label       category
0  225794  Non-invasive Ventilation  2-Ventilation
1  227577                BiPap Mode    Respiratory
2  227578                BiPap Mask    Respiratory
3  227583              Autoset/CPAP    Respiratory
4  227579                BiPap EPAP    Respiratory
5  227580                BiPap IPAP    Respiratory
6  227581  BiPap bpm (S/T -Back up)    Respiratory
7  227582             BiPap O2 Flow    Respiratory


In [ ]:
niv_query = """
SELECT subject_id, stay_id, charttime AS niv_end
FROM `physionet-data.mimiciv_icu.chartevents`
WHERE itemid IN (225794, 227577, 227583, 227579, 227580, 227582)
"""
niv_df = client.query(niv_query).to_dataframe()

In [ ]:
niv_df.head()

,subject_id,stay_id,niv_end
0,16088475,31413377,2186-09-03 01:00:00
1,16578832,33938469,2159-12-11 06:00:00
2,14990450,37053539,2179-03-04 23:00:00
3,17150899,36558687,2144-03-08 08:00:00
4,19826427,37559521,2117-05-20 05:00:00


In [ ]:
# If NIV more than 48 hours after extubation_time, then patient has extubation failure

from datetime import timedelta

for index, row in vent_outcomes.iterrows():
  subject_id = row['subject_id']
  extubation_time = row['extubation_time']
  time_window_end = extubation_time + timedelta(hours=48)

  niv_events = niv_df[
      (niv_df['subject_id'] == subject_id) &
      (niv_df['niv_end'] > extubation_time) &
      (niv_df['niv_end'] > time_window_end)
  ]

  if not niv_events.empty:
    vent_outcomes.at[index, 'niv_48'] = 1

vent_outcomes.head()

,subject_id,hadm_id,stay_id,hadm_no,stay_no,intubation_time,ventilation_starttime,ventilation_endtime,ventilation_time,extubation_time,patient_weight,deathtime,los,mortality,died_on_vent,ventend_death,vent_duration_mins,age,niv_48
0,10001884,26184834,37510196,1,1,2131-01-11 04:31:00,2131-01-11 04:40:00,2131-01-12 17:40:00,2220.0,2131-01-12 17:41:00,65.0,2131-01-20 05:15:00,17796,1,0,NaN,2230.0,68,NaN
1,10001884,26184834,37510196,1,1,NaT,2131-01-13 04:00:00,2131-01-19 17:45:00,9465.0,NaT,65.0,2131-01-20 05:15:00,17796,1,1,690.0,10155.0,68,NaN
2,10002428,28662225,38875437,1,1,NaT,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,2156-04-22 17:11:00,43.0,NaT,24610,0,0,NaN,4141.0,80,NaN
3,10003400,23559586,34577403,1,1,NaT,2137-08-10 20:33:00,2137-08-11 13:53:00,1040.0,NaT,NaN,2137-09-02 17:05:00,42778,1,1,31872.0,32912.0,72,NaN
4,10003637,28317408,32824762,1,1,NaT,2150-05-20 07:45:00,2150-05-22 18:39:00,3534.0,NaT,88.0,2150-05-22 16:25:00,11314,1,1,-134.0,3400.0,57,NaN


In [ ]:
# Second, reintubation within 48h of extubation

for index, row in vent_outcomes.iterrows():
  subject_id = row['subject_id']
  extubation_time = row['extubation_time']
  time_window_end = extubation_time + timedelta(hours=48)

  reintubations = vent_outcomes[
      (vent_outcomes['subject_id'] == subject_id) &
      (vent_outcomes['intubation_time'] > extubation_time) &
      (vent_outcomes['intubation_time'] <= time_window_end)
  ]

  if not reintubations.empty:
    vent_outcomes.at[index, 'reintubation_48'] = 1

vent_outcomes.head()

,subject_id,hadm_id,stay_id,hadm_no,stay_no,intubation_time,ventilation_starttime,ventilation_endtime,ventilation_time,extubation_time,patient_weight,deathtime,los,mortality,died_on_vent,ventend_death,vent_duration_mins,age,niv_48,reintubation_48
0,10001884,26184834,37510196,1,1,2131-01-11 04:31:00,2131-01-11 04:40:00,2131-01-12 17:40:00,2220.0,2131-01-12 17:41:00,65.0,2131-01-20 05:15:00,17796,1,0,NaN,2230.0,68,NaN,NaN
1,10001884,26184834,37510196,1,1,NaT,2131-01-13 04:00:00,2131-01-19 17:45:00,9465.0,NaT,65.0,2131-01-20 05:15:00,17796,1,1,690.0,10155.0,68,NaN,NaN
2,10002428,28662225,38875437,1,1,NaT,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,2156-04-22 17:11:00,43.0,NaT,24610,0,0,NaN,4141.0,80,NaN,NaN
3,10003400,23559586,34577403,1,1,NaT,2137-08-10 20:33:00,2137-08-11 13:53:00,1040.0,NaT,NaN,2137-09-02 17:05:00,42778,1,1,31872.0,32912.0,72,NaN,NaN
4,10003637,28317408,32824762,1,1,NaT,2150-05-20 07:45:00,2150-05-22 18:39:00,3534.0,NaT,88.0,2150-05-22 16:25:00,11314,1,1,-134.0,3400.0,57,NaN,NaN


In [ ]:
vent_outcomes.head(50)

,subject_id,hadm_id,stay_id,hadm_no,stay_no,intubation_time,ventilation_starttime,ventilation_endtime,ventilation_time,extubation_time,patient_weight,deathtime,los,mortality,died_on_vent,ventend_death,vent_duration_mins,age,niv_48,reintubation_48
0,10001884,26184834,37510196,1,1,2131-01-11 04:31:00,2131-01-11 04:40:00,2131-01-12 17:40:00,2220.0,2131-01-12 17:41:00,65.0,2131-01-20 05:15:00,17796,1,0,NaN,2230.0,68,NaN,NaN
1,10001884,26184834,37510196,1,1,NaT,2131-01-13 04:00:00,2131-01-19 17:45:00,9465.0,NaT,65.0,2131-01-20 05:15:00,17796,1,1,690.0,10155.0,68,NaN,NaN
2,10002428,28662225,38875437,1,1,NaT,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,2156-04-22 17:11:00,43.0,NaT,24610,0,0,NaN,4141.0,80,NaN,NaN
3,10003400,23559586,34577403,1,1,NaT,2137-08-10 20:33:00,2137-08-11 13:53:00,1040.0,NaT,NaN,2137-09-02 17:05:00,42778,1,1,31872.0,32912.0,72,NaN,NaN
4,10003637,28317408,32824762,1,1,NaT,2150-05-20 07:45:00,2150-05-22 18:39:00,3534.0,NaT,88.0,2150-05-22 16:25:00,11314,1,1,-134.0,3400.0,57,NaN,NaN
5,10004235,24181354,34100191,1,1,NaT,2196-02-24 16:52:00,2196-02-27 16:28:00,4296.0,2196-02-27 16:29:00,127.0,NaT,12924,0,0,NaN,4297.0,47,NaN,NaN
6,10004401,25777141,39699336,1,1,NaT,2144-06-05 20:45:00,2144-06-18 16:12:00,18447.0,NaT,NaN,2144-06-18 21:30:00,18825,1,1,318.0,18765.0,82,NaN,NaN
7,10004606,29242151,30213599,1,1,NaT,2159-02-20 18:04:00,2159-02-22 11:04:00,2460.0,2159-02-22 11:04:00,55.0,NaT,20348,0,0,NaN,2460.0,64,NaN,NaN
8,10004720,22081550,35009126,1,1,NaT,2186-11-12 20:29:00,2186-11-17 14:00:00,6811.0,2186-11-17 14:01:00,70.0,2186-11-17 18:30:00,7229,1,0,NaN,6812.0,61,NaN,NaN
9,10004733,27411876,39635619,1,1,NaT,2174-12-04 12:25:00,2174-12-07 16:20:00,4555.0,2174-12-07 16:21:00,112.5,NaT,33272,0,0,NaN,4556.0,51,NaN,NaN


In [ ]:
# Third, died within 48h of extubation

for index, row in vent_outcomes.iterrows():
  subject_id = row['subject_id']
  extubation_time = row['extubation_time']
  time_window_end = extubation_time + timedelta(hours=48)

  died48 = vent_outcomes[
      (vent_outcomes['subject_id'] == subject_id) &
      (vent_outcomes['deathtime'] > extubation_time) &
      (vent_outcomes['deathtime'] <= time_window_end)
  ]

  if not died48.empty:
    vent_outcomes.at[index, 'died_48'] = 1

vent_outcomes.head()

,subject_id,hadm_id,stay_id,hadm_no,stay_no,intubation_time,ventilation_starttime,ventilation_endtime,ventilation_time,extubation_time,...,deathtime,los,mortality,died_on_vent,ventend_death,vent_duration_mins,age,niv_48,reintubation_48,died_48
0,10001884,26184834,37510196,1,1,2131-01-11 04:31:00,2131-01-11 04:40:00,2131-01-12 17:40:00,2220.0,2131-01-12 17:41:00,...,2131-01-20 05:15:00,17796,1,0,NaN,2230.0,68,NaN,NaN,NaN
1,10001884,26184834,37510196,1,1,NaT,2131-01-13 04:00:00,2131-01-19 17:45:00,9465.0,NaT,...,2131-01-20 05:15:00,17796,1,1,690.0,10155.0,68,NaN,NaN,NaN
2,10002428,28662225,38875437,1,1,NaT,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,2156-04-22 17:11:00,...,NaT,24610,0,0,NaN,4141.0,80,NaN,NaN,NaN
3,10003400,23559586,34577403,1,1,NaT,2137-08-10 20:33:00,2137-08-11 13:53:00,1040.0,NaT,...,2137-09-02 17:05:00,42778,1,1,31872.0,32912.0,72,NaN,NaN,NaN
4,10003637,28317408,32824762,1,1,NaT,2150-05-20 07:45:00,2150-05-22 18:39:00,3534.0,NaT,...,2150-05-22 16:25:00,11314,1,1,-134.0,3400.0,57,NaN,NaN,NaN


In [ ]:
total_niv_48 = vent_outcomes['niv_48'].sum()

print(f"Total patients requiring NIV > 48h after extubation: {total_niv_48}")

Total patients requiring NIV > 48h after extubation: 662.0


In [ ]:
total_reintubation = vent_outcomes['reintubation_48'].sum()

print(f"Total patients requiring reintubation < 48h after extubation: {total_reintubation}")

Total patients requiring reintubation < 48h after extubation: 5.0


In [ ]:
total_died_48 = vent_outcomes['died_48'].sum()

print(f"Total patients that died < 48h after extubation: {total_died_48}")

Total patients that died < 48h after extubation: 1150.0


In [ ]:
# Define ext_success as 1 if all failure indicators are 0 or NaN AND extubation_time is not NaT
vent_outcomes['ext_success'] = (
    (vent_outcomes['extubation_time'].notna()) &  # New condition added here
    ((vent_outcomes['died_on_vent'].isna()) | (vent_outcomes['died_on_vent'] == 0)) &
    ((vent_outcomes['niv_48'].isna()) | (vent_outcomes['niv_48'] == 0)) &
    ((vent_outcomes['reintubation_48'].isna()) | (vent_outcomes['reintubation_48'] == 0)) &
    ((vent_outcomes['died_48'].isna()) | (vent_outcomes['died_48'] == 0))
).astype(int)

In [ ]:
vent_outcomes.head(50)

,subject_id,hadm_id,stay_id,hadm_no,stay_no,intubation_time,ventilation_starttime,ventilation_endtime,ventilation_time,extubation_time,...,los,mortality,died_on_vent,ventend_death,vent_duration_mins,age,niv_48,reintubation_48,died_48,ext_success
0,10001884,26184834,37510196,1,1,2131-01-11 04:31:00,2131-01-11 04:40:00,2131-01-12 17:40:00,2220.0,2131-01-12 17:41:00,...,17796,1,0,NaN,2230.0,68,NaN,NaN,NaN,1
1,10001884,26184834,37510196,1,1,NaT,2131-01-13 04:00:00,2131-01-19 17:45:00,9465.0,NaT,...,17796,1,1,690.0,10155.0,68,NaN,NaN,NaN,0
2,10002428,28662225,38875437,1,1,NaT,2156-04-19 20:10:00,2156-04-22 17:05:00,4135.0,2156-04-22 17:11:00,...,24610,0,0,NaN,4141.0,80,NaN,NaN,NaN,1
3,10003400,23559586,34577403,1,1,NaT,2137-08-10 20:33:00,2137-08-11 13:53:00,1040.0,NaT,...,42778,1,1,31872.0,32912.0,72,NaN,NaN,NaN,0
4,10003637,28317408,32824762,1,1,NaT,2150-05-20 07:45:00,2150-05-22 18:39:00,3534.0,NaT,...,11314,1,1,-134.0,3400.0,57,NaN,NaN,NaN,0
5,10004235,24181354,34100191,1,1,NaT,2196-02-24 16:52:00,2196-02-27 16:28:00,4296.0,2196-02-27 16:29:00,...,12924,0,0,NaN,4297.0,47,NaN,NaN,NaN,1
6,10004401,25777141,39699336,1,1,NaT,2144-06-05 20:45:00,2144-06-18 16:12:00,18447.0,NaT,...,18825,1,1,318.0,18765.0,82,NaN,NaN,NaN,0
7,10004606,29242151,30213599,1,1,NaT,2159-02-20 18:04:00,2159-02-22 11:04:00,2460.0,2159-02-22 11:04:00,...,20348,0,0,NaN,2460.0,64,NaN,NaN,NaN,1
8,10004720,22081550,35009126,1,1,NaT,2186-11-12 20:29:00,2186-11-17 14:00:00,6811.0,2186-11-17 14:01:00,...,7229,1,0,NaN,6812.0,61,NaN,NaN,1.0,0
9,10004733,27411876,39635619,1,1,NaT,2174-12-04 12:25:00,2174-12-07 16:20:00,4555.0,2174-12-07 16:21:00,...,33272,0,0,NaN,4556.0,51,NaN,NaN,NaN,1


In [ ]:
num_extubations = vent_outcomes['extubation_time'].notna().sum()
print(f"Number of extubation_time entries: {num_extubations}")

Number of extubation_time entries: 9090


In [ ]:
total_successful_extubations = vent_outcomes['ext_success'].sum()

print(f"Total successful extubations: {total_successful_extubations}")

Total successful extubations: 7274


In [ ]:
num_unique_subjects = vent_outcomes['subject_id'].nunique()
print(f"Number of unique subject_id values: {num_unique_subjects}")

Number of unique subject_id values: 10072


In [ ]:
num_unique_admissions = vent_outcomes['stay_id'].nunique()
print(f"Number of unique admissions: {num_unique_admissions}")

Number of unique admissions: 10204


In [ ]:
final = vent_outcomes.copy()

In [ ]:
final.to_csv('final.csv', index=False)

In [ ]:
from google.colab import files
files.download('final.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Get a distinct list of patient keys from ventilation_df
ventilation_keys = ventilation_ad_filtered[['subject_id', 'stay_id']].drop_duplicates()
print(f"Unique ventilation subject/stay pairs: {len(ventilation_keys)}")

Unique ventilation subject/stay pairs: 31059


In [ ]:
project_id = "mimic-ventilation-project"
client = bigquery.Client(project=project_id)

# create temp_dataset
dataset_id = "temp_dataset"
dataset_ref = bigquery.Dataset(f"{project_id}.{dataset_id}")
dataset_ref.location = "US"

# create the dataset
client.create_dataset(dataset_ref, exists_ok=True)
print(f"Dataset created or already exists: {project_id}.{dataset_id}")

# upload ventilation_keys to BigQuery as a temporary table
table_id = f"{project_id}.{dataset_id}.ventilation_keys"

job = client.load_table_from_dataframe(
    ventilation_keys,
    table_id,
    job_config=bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")
)
job.result()
print(f"Uploaded {len(ventilation_keys)} ventilation keys to BigQuery table: {table_id}")

chartevents_query = f"""
SELECT ce.*
FROM `physionet-data.mimiciv_icu.chartevents` ce
JOIN `{table_id}` vk
ON ce.subject_id = vk.subject_id AND ce.stay_id = vk.stay_id
LIMIT 10000
"""

chartevents_df = client.query(chartevents_query).to_dataframe()
print(f"Retrieved {len(chartevents_df)} matching chartevents rows.")
chartevents_df.head()

Dataset created or already exists: mimic-ventilation-project.temp_dataset
Uploaded 31059 ventilation keys to BigQuery table: mimic-ventilation-project.temp_dataset.ventilation_keys
Retrieved 10000 matching chartevents rows.


,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
0,11573883,22286474,38278056,36832,2166-04-02 04:00:00,2166-04-02 06:02:00,224000,Patent,NaN,None,0
1,17417573,24710404,36358936,78217,2208-07-22 04:00:00,2208-07-22 05:51:00,224000,Patent,NaN,None,0
2,10772942,23525346,31145789,3459,2175-08-29 20:11:00,2175-08-29 20:12:00,224000,Patent,NaN,None,0
3,15890451,25916203,36337702,65712,2139-06-22 08:00:00,2139-06-22 07:28:00,224000,Patent,NaN,None,0
4,16800170,21622732,38337923,6253,2192-09-22 00:00:00,2192-09-21 23:33:00,224000,Patent,NaN,None,0


In [ ]:
chartevents_query = f"""
SELECT ce.*
FROM `physionet-data.mimiciv_icu.chartevents` ce
JOIN `{table_id}` vk
  ON ce.subject_id = vk.subject_id AND ce.stay_id = vk.stay_id
WHERE ce.itemid = 223849
"""

# Run the query and load into a DataFrame
chartevents_ventmode = client.query(chartevents_query).to_dataframe()
print(f"Retrieved {len(chartevents_ventmode)} charted ventilator mode rows.")
chartevents_ventmode.head()

Retrieved 447800 charted ventilator mode rows.


,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
0,19213007,27273230,32477673,55751,2114-02-12 14:00:00,2114-02-12 14:01:00,223849,CMV,1.0,None,0
1,11000360,25315023,37277230,66037,2170-05-02 00:00:00,2170-05-02 00:09:00,223849,CMV,1.0,None,0
2,11340773,26496522,38661600,29398,2148-08-18 08:00:00,2148-08-18 08:04:00,223849,APRV,26.0,None,0
3,13102401,22251975,31004936,92018,2148-06-05 01:34:00,2148-06-05 01:36:00,223849,APRV,26.0,None,0
4,19406079,25955519,30591925,95841,2139-01-19 04:00:00,2139-01-19 04:22:00,223849,APRV,26.0,None,0


In [ ]:
# rename and convert charttime to datetime
chartevents_ventmode = chartevents_ventmode.rename(columns={
    'value': 'mode',
    'valuenum': 'mode_code'
})
chartevents_ventmode['charttime'] = pd.to_datetime(chartevents_ventmode['charttime'])

# keep only required columns
chartevents_ventmode = chartevents_ventmode[['subject_id', 'hadm_id', 'stay_id', 'charttime', 'mode', 'mode_code']]

# merge with ventilation_ad_filtered on subject_id and stay_id, keeping all unique columns
ventilation_merged = pd.merge(
    ventilation_ad_filtered,
    chartevents_ventmode,
    on=['subject_id', 'hadm_id', 'stay_id'],
    how='inner',  # change to 'outer' or 'left' if needed
    suffixes=('_ventad', '_chart')
)

# sort by subject_id, hadm_id, stay_id and times chronologically
ventilation_merged = ventilation_merged.sort_values(
    by=['subject_id', 'hadm_id', 'stay_id', 'starttime', 'endtime', 'charttime']
).reset_index(drop=True)

In [ ]:
ventilation_merged.head(50)

,subject_id,hadm_id,stay_id,starttime,endtime,value,itemid,patientweight,admittime,dischtime,deathtime,los,mortality,charttime,mode,mode_code
0,10002013,23581541,39060235,2160-05-18 14:19:00,2160-05-18 18:01:00,222.0,225792,96.0,2160-05-18 07:45:00,2160-05-23 13:30:00,NaT,7545,0,2160-05-18 14:00:00,CMV/ASSIST,2.0
1,10002013,23581541,39060235,2160-05-18 14:19:00,2160-05-18 18:01:00,222.0,225792,96.0,2160-05-18 07:45:00,2160-05-23 13:30:00,NaT,7545,0,2160-05-18 15:00:00,CMV/ASSIST/AutoFlow,49.0
2,10002013,23581541,39060235,2160-05-18 14:19:00,2160-05-18 18:01:00,222.0,225792,96.0,2160-05-18 07:45:00,2160-05-23 13:30:00,NaT,7545,0,2160-05-18 16:00:00,CPAP/PSV,11.0
3,10002013,23581541,39060235,2160-05-18 18:00:00,2160-05-18 18:01:00,1.0,227194,96.0,2160-05-18 07:45:00,2160-05-23 13:30:00,NaT,7545,0,2160-05-18 14:00:00,CMV/ASSIST,2.0
4,10002013,23581541,39060235,2160-05-18 18:00:00,2160-05-18 18:01:00,1.0,227194,96.0,2160-05-18 07:45:00,2160-05-23 13:30:00,NaT,7545,0,2160-05-18 15:00:00,CMV/ASSIST/AutoFlow,49.0
5,10002013,23581541,39060235,2160-05-18 18:00:00,2160-05-18 18:01:00,1.0,227194,96.0,2160-05-18 07:45:00,2160-05-23 13:30:00,NaT,7545,0,2160-05-18 16:00:00,CPAP/PSV,11.0
6,10002428,23473524,35479615,2156-05-11 16:05:00,2156-05-20 10:45:00,12640.0,225792,48.4,2156-05-11 14:49:00,2156-05-22 14:16:00,NaT,15807,0,2156-05-11 16:00:00,CMV/ASSIST,2.0
7,10002428,23473524,35479615,2156-05-11 16:05:00,2156-05-20 10:45:00,12640.0,225792,48.4,2156-05-11 14:49:00,2156-05-22 14:16:00,NaT,15807,0,2156-05-11 20:00:00,CMV/ASSIST/AutoFlow,49.0
8,10002428,23473524,35479615,2156-05-11 16:05:00,2156-05-20 10:45:00,12640.0,225792,48.4,2156-05-11 14:49:00,2156-05-22 14:16:00,NaT,15807,0,2156-05-12 00:00:00,CMV/ASSIST/AutoFlow,49.0
9,10002428,23473524,35479615,2156-05-11 16:05:00,2156-05-20 10:45:00,12640.0,225792,48.4,2156-05-11 14:49:00,2156-05-22 14:16:00,NaT,15807,0,2156-05-12 05:00:00,CMV/ASSIST/AutoFlow,49.0


In [ ]:
test = chartevents_with_outcomes_sorted

NameError: name 'chartevents_with_outcomes_sorted' is not defined

In [ ]:
unique_value_pairs = (
    test[['value', 'valuenum']]
    .dropna(subset=['value', 'valuenum'])
    .drop_duplicates()
    .sort_values(by=['value', 'valuenum'])
    .reset_index(drop=True)
)

print("Unique (value, valuenum) pairs:")
print(unique_value_pairs)

Unique (value, valuenum) pairs:
                  value  valuenum
0                  APRV      26.0
1     Apnea Ventilation      17.0
2                   CMV       1.0
3            CMV/ASSIST       2.0
4   CMV/ASSIST/AutoFlow      49.0
5          CMV/AutoFlow      48.0
6                  CPAP      10.0
7              CPAP/PPS      53.0
8              CPAP/PSV      11.0
9                   MMV      12.0
10         MMV/AutoFlow      50.0
11              MMV/PSV      13.0
12     MMV/PSV/AutoFlow      51.0
13                 PCV+      14.0
14             PCV+/PSV      45.0
15           PCV+Assist      71.0
16                 SIMV       6.0
17        SIMV/AutoFlow      46.0
18             SIMV/PSV       7.0
19    SIMV/PSV/AutoFlow      47.0
20      SYNCHRON MASTER      15.0
21       SYNCHRON SLAVE      16.0
22              Standby      30.0


In [ ]:
# define valuenum codes for each group based on categorisation
control_modes = {26.0, 1.0, 2.0, 48.0, 14.0, 45.0, 71.0, 49.0, 17.0}
support_modes = {10.0, 53.0, 11.0, 12.0, 50.0, 13.0, 51.0, 6.0, 46.0, 7.0, 47.0, 30.0}

# function to classify ventilation type
def classify_vent_mode(val):
    if val in control_modes:
        return "Control"
    elif val in support_modes:
        return "Support"
    else:
        return "Other"  # For unclassified modes like Apnea, Standby, SYNCHRON, etc.

# apply classification
test['ventilation_type'] = chartevents_ventmode_sorted['valuenum'].apply(classify_vent_mode)

# preview the updated DataFrame
test[['subject_id', 'stay_id', 'charttime', 'value', 'valuenum', 'ventilation_type', 'died_in_hospital']].head(50)

,subject_id,stay_id,charttime,value,valuenum,ventilation_type,died_in_hospital
109066,10000980,39765666,2189-06-27 09:00:00,CPAP/PSV,11.0,Support,False
147814,10000980,39765666,2189-06-27 10:00:00,Standby,30.0,Support,False
318407,10002013,39060235,2160-05-18 14:00:00,CMV/ASSIST,2.0,Control,False
448361,10002013,39060235,2160-05-18 15:00:00,CMV/ASSIST/AutoFlow,49.0,Support,False
284571,10002013,39060235,2160-05-18 16:00:00,CPAP/PSV,11.0,Control,False
150285,10002155,32358465,2131-03-10 00:14:00,CPAP/PSV,11.0,Other,True
254708,10002155,32358465,2131-03-10 04:00:00,Standby,30.0,Support,True
182395,10002428,34807493,2156-04-30 22:00:00,CPAP/PSV,11.0,Control,False
28604,10002428,34807493,2156-05-01 10:00:00,CPAP/PSV,11.0,Control,False
430760,10002428,34807493,2156-05-02 04:00:00,CPAP/PSV,11.0,Support,False


In [ ]:
# get first charttime per stay
first_times = (
    test
    .groupby(['subject_id', 'stay_id'], as_index=False)['charttime']
    .min()
    .rename(columns={'charttime': 'first_charttime'})
)

In [ ]:
# merge to compute relative time in hours
df = test.merge(first_times, on=['subject_id', 'stay_id'], how='left')
df['hours_since_start'] = (df['charttime'] - df['first_charttime']).dt.total_seconds() / 3600
df['hour_bin'] = df['hours_since_start'].astype(int)

In [ ]:
# drop duplicate states per hour bin per patient
df = df.drop_duplicates(subset=['subject_id', 'stay_id', 'hour_bin'])

# create a synthetic patient ID for visualization
df['patient_id'] = df['subject_id'].astype(str) + '_' + df['stay_id'].astype(str)

# keep only columns needed for plotting
vent_long = df[['patient_id', 'hour_bin', 'ventilation_type', 'died_in_hospital']].copy()

In [ ]:
print(vent_long.head(50))

           patient_id  hour_bin ventilation_type  died_in_hospital
0   10000980_39765666         0          Support             False
1   10000980_39765666         1          Support             False
2   10002013_39060235         0          Control             False
3   10002013_39060235         1          Support             False
4   10002013_39060235         2          Control             False
5   10002155_32358465         0            Other              True
6   10002155_32358465         3          Support              True
7   10002428_34807493         0          Control             False
8   10002428_34807493        12          Control             False
9   10002428_34807493        30          Support             False
10  10002428_34807493        31          Control             False
11  10002428_35479615         0          Support             False
12  10002428_35479615         4          Control             False
13  10002428_35479615         8          Support             F

In [ ]:
import plotly.graph_objects as go
import numpy as np

# Sample structure of vent_long (make sure ventilation_type and died_in_hospital are lowercase and bool)
vent_long['ventilation_type'] = vent_long['ventilation_type'].str.lower()
vent_long['died_in_hospital'] = vent_long['died_in_hospital'].astype(bool)

# Step 1: Extract ventilation mode at hour 0 for each patient
vent_mode_start = vent_long[vent_long['hour_bin'] == 0][['patient_id', 'ventilation_type']]

# Step 2: Extract hospital outcome per patient (died_in_hospital)
outcome = vent_long.groupby('patient_id')['died_in_hospital'].last().reset_index()

# Step 3: Merge for Sankey source and target labels
df_sankey = vent_mode_start.merge(outcome, on='patient_id')

# Define node labels (left and right sides)
node_labels = ['control', 'support', 'died', 'survived']

# Map labels to indices for source and target
source_map = {'control': 0, 'support': 1}
target_map = {True: 2, False: 3}

# Count flows (number of patients in each source->target pair)
flows = df_sankey.groupby(['ventilation_type', 'died_in_hospital']).size().reset_index(name='count')

# Create sankey data
sources = flows['ventilation_type'].map(source_map)
targets = flows['died_in_hospital'].map(target_map)
values = flows['count']

# Define colors for nodes and links
node_colors = ['darkblue', 'lightblue', 'red', 'green']
link_colors = []

for _, row in flows.iterrows():
    # Color links same as source ventilation type but lighter for visual
    if row['ventilation_type'] == 'control':
        color = 'rgba(0,0,139,0.6)'  # dark blue semi-transparent
    else:
        color = 'rgba(173,216,230,0.6)'  # light blue semi-transparent

    # Override link color if outcome is death (red) or survive (green)
    if row['died_in_hospital']:
        color = 'rgba(255,0,0,0.6)'  # red
    else:
        color = 'rgba(0,128,0,0.6)'  # green
    link_colors.append(color)

# Create Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=node_labels,
        color=node_colors
    ),
    link=dict(
        source=sources,
        target=targets,
        value=values,
        color=link_colors
    )
)])

fig.update_layout(title_text="Ventilation Mode to Hospital Outcome Sankey Plot", font_size=14)
fig.show()